Multi-Output Regression Module
=============================

This is an example notebook detailing the functions currently contained in this module for Multi-Output (or Multi-Task) Regression. These include:

1. Network structure discovery in continuous domains for modeling relations and feature selection.
2. Combine several sampled networks to create an empirical approximation of the posterior distribution over network structures. This allows for a more robust estimation, especially when few samples are available.
3. Use of the learned structure to fit a Multi-Task Regression model that can perform regression over several target variables simultaneously.

In [ ]:
import numpy as np
from numpy.random import RandomState

rng = RandomState(1802)

The first step is to load some data. In this example we will generate some random data from a Gaussian Network with fixed, dummy parameters:

In [ ]:
from core.gaussian import sample_from_gn
from structure.graph_generation import random_mbc
from structure.graphs import plot_digraph

# We use 15 variables, 10 features amd 5 targets
n_variables = 15
n_features, n_targets = 10, 5
variables = list(range(n_variables))

# Data generation parameters
n_samples = 200
test_samples = 100

# the mean, variance, and weight of the interaction
gen_mean = np.zeros(n_variables)
gen_var = np.zeros(n_variables) + 0.2
gen_weight = 2

# sample a random structure with a given fan in to obtain a sparse graph
fan_in = 5
graph = random_mbc(n_features, n_targets, rng=rng, fan_in=fan_in)

# Generate the data from the parameters
data = sample_from_gn(graph, gen_mean, gen_var, gen_weight, n_samples + test_samples, rng)

training_data, test_data = data[:n_samples], data[n_samples:]

# Plot the digraph
plot_digraph(graph)

To obtain samples of DAG structures we will use MCMC sampling. To this end we need a state sapce, a proposal that defines the neighborhood of the Directed Acyclic Graphs (DAGs) in this space and score function to determine how good these structures are given the training data

In [ ]:
from metrics.score import BGe
from structure.graphs import plot_digraph
from mcmc.graphs.sampler import MHStructureSampler
from mcmc.graphs.proposal import MBCProposal, basic_move, rev_move, nbhr_move

# First we define the proposal. We will use three moves: Basica addition and removal of edges,
# the REV move and NBHR move, with given probabilities of being used
moves = [basic_move, rev_move, nbhr_move]
move_probs = [13/15, 1/15, 1/15]

# The MBCProposal already incorportates the necessary constraints on the graphs. The BGe is a
# metric specifically designed for scoring Gaussian Networks and it is the one implemented here
proposal = MBCProposal(moves, move_prob=move_probs, score=BGe, fan_in=5, random_state=rng)

# Initialize a structure sampler using Metropolis-Hastings with the above defined proposal.
sampler = MHStructureSampler(
    proposal=proposal, n_steps=10000, sample_freq=100, burn_in=5000, verbose=True, rng=rng
)

At this point we can collect samples by simply running the sampler with our training data: 

In [ ]:
X, y = training_data[:, :n_features], training_data[:, n_features:]

# If return scores is true, the method will return a tuple of networks and 
# their respective scores according to the used metric (in this case BGe)
trace = sampler.generate_samples((X, y), return_scores=True)
samples, scores = trace

With these samples we can create an empiracal distribution over networks and check the (conditional) probability of edges:

In [ ]:
from mcmc.graphs.sampler import DAGDistribution
dist = DAGDistribution(samples)

print('The marginal probability of edge 0 --> 1 is {0}'.format(dist.edge_prob((0, 1))))
print('The marginal probability of edge 0 --> 1 given edge 2 --> 3 is present in the graph is {0}'
      .format(dist.edge_conditional_prob((0, 1), (2, 3)))
)

Finally, we can train a Multi-output regression model called Multi-ouput Gaussian Network Regressor Ensamble using these sampled netowkrs:

In [ ]:
from model.mgnr import MGNREnsemble

# We initialize the model with the number of networks we want to combine and the optimizer we want to use in case
# samples are not passed as an argument during the fitting procedure (in this case we use the computed samples)
model = MGNREnsemble(k=100, optimizer=sampler, rng=rng, verbose=True).fit(X, y, samples)

This model can be tested using the test samples we generated above:

In [ ]:
X_test, y_test = test_data[:, :n_features], test_data[:, n_features:]

sq_error = 0
for x, y in zip(X_test, y_test):
    predicted = model.predict(X).reshape(1, -1)

    sq_error += (y - predicted) ** 2

rmse = np.sqrt(sq_error / X_test.shape[0])
print('The RMSE error was: {:.2f}'.format(rmse))